### INSTITUTO FEDERAL DE SÃO PAULO - Bragança Paulista SP
### Curso: Análise e Desenvolvimento de Sistemas
### Disciplina: Análise de Dados - BRAADAD
### Mini-Projeto 1
### Prof. César A. S. Lima, Dr.

### Projeto 1 - Prevendo a Ocorrência de Crises Epiléticas

### Etapa 1 - Definição ou Compreenção do Problema de Negócio

A epilepsia é um distúrbio do sistema nervoso central (SNC), afetando cerca de 1,2% (3,4 milhões de pessoas) nos EUA e mais de 65 milhões em todo o mundo. Além disso, cerca de 1 em cada 26 pessoas desenvolverá epilepsia em algum momento da vida. Existem muitos tipos de convulsões, cada uma com sintomas diferentes, como perda de consciência, movimentos bruscos ou confusão. 

Algumas convulsões são muito mais difíceis de detectar visualmente, os pacientes geralmente apresentam sintomas como não responder ou olhar sem expressão por um breve período de tempo. As convulsões podem ocorrer inesperadamente e podem resultar em lesões como queda, mordedura da língua ou perda do controle da urina ou fezes. Portanto, essas são algumas das razões pelas quais a detecção de convulsões é de extrema importância para pacientes sob supervisão médica que se suspeitem estar propensos a convulsões.

Este projeto usará métodos de <b>classificação binária</b> para prever se um indivíduo está tendo uma convulsão em algum momento, ou não.

<b>Objetivo</b>: Prever se um paciente está tendo uma convulsão ou não através de 178 leituras de EEG (Eletroencefalograma) por segundo.

Como métrica de avaliação do modelo usaremos a <b>AUC Score (Area Under The Curve Score)</b>, cujo valor vai de 1 a 100% e para esse problema o valor da métrica deve ser aproximadamente de 99%, uma vez que a previsão do modelo está relacionada a casos de vida ou morte. Usaremos a métrica calculada no dataset de validação.

A AUC Score é particularmente útil e recomendada em várias situações em especial nos modelos de classificação binária que será este caso. A AUC Score é particularmente útil e recomendada nas seguintes situações:

Modelos de Classificação Binária: É sua principal aplicação, onde o objetivo é classificar instâncias em duas classes (por exemplo, "doente" ou "saudável", "spam" ou "não spam").

Conjuntos de Dados Desbalanceados: Quando uma classe tem muito mais exemplos do que a outra (por exemplo, 95% de casos negativos e 5% de casos positivos), métricas como a acurácia podem ser enganosas. Um modelo que sempre prevê a classe majoritária pode ter alta acurácia, mas ser inútil. A AUC é robusta a esse desbalanceamento, pois avalia o desempenho do modelo em todos os limiares de classificação, considerando tanto os verdadeiros positivos quanto os falsos positivos.

Comparação de Modelos: A AUC é uma excelente métrica para comparar o desempenho de diferentes modelos de classificação. O modelo com a maior AUC geralmente é considerado o melhor, pois demonstra uma capacidade superior de distinguir entre as classes.

## Etapa 2 - Compreensão dos Dados

O conjunto de dados está disponível no repositório de aprendizado de máquina do Kaggle e em anexo a este Jupyter Notebook.

https://www.kaggle.com/datasets/harunshimanto/epileptic-seizure-recognition

O conjunto de dados original da referência consiste em 5 pastas diferentes, cada uma com 100 arquivos, cada um representando um único sujeito/pessoa. Cada arquivo é uma gravação da atividade cerebral por 23,6 segundos. A série temporal correspondente é amostrada em 4.097 pontos de dados. Cada ponto de dados é o valor da gravação do EEG em um momento diferente. Portanto, temos um total de 500 indivíduos, cada um com 4.097 pontos de dados por 23,5 segundos.

Dividimos e embaralhamos cada cada um dos 4.097 pontos de dados em 23 blocos, cada bloco contendo 178 pontos de dados por 1 segundo, e cada ponto de dados é o valor do registro de EEG em um momento diferente. Portanto, agora temos 23 x 500 = 11.500 informações (linha), cada informação contendo 178 pontos de dados por 1 segundo (coluna), e a última coluna representa o rótulo y {1,2,3,4,5}.

A variável de resposta é y na coluna 179, as variáveis explicativas X1, X2, …, X178

### Análise Exploratória - AED

In [1]:
%pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.0 MB 10.1 MB/s eta 0:00:01
   --------------- ------------------------ 4.2/11.0 MB 11.0 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 11.3 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 11.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Carregando os dados
import pandas as pd
# Um DataFrame é um objeto do pandas que faz a leitura de 
# arquivos do excel e os converte para um objeto capaz de exibir
# os dados como se fossem uma planilha.
dataset = pd.read_csv("dados_originais.csv")

In [4]:
# Visualizando alguns registros
# As colunas de x1 a x178 registram a leitura de 1 segundo de frequencia do Eletroencefalograma.
# Cada pessoa possui 23 segundos. Então,cada paciente possui 23 linhas na planilha.
# São 500 pessoas, então temos um total de 11.500 linhas na planilha.
# A coluna y registram valores de 1 a 5, sendo que, apenas o valor 1 representa o paciente que
# teve convulsão. Os valores de 2 a 5 são sintomas, mas, os pacientes que receberam estes valores
# não manifetaram a doença. 
# A coluna unnamed:0 não significa nada.
# O método head() do objeto dataset exibe apenas as 5 primeiras linhas do arquivo.
dataset.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


Vamos criar uma coluna chamada Diagnostico em que 1 é quando um paciente está tendo uma convulsão e 0 é quando um paciente não está tendo uma convulsão.

Na última coluna, somente o valor 1 representa convulsão. Os demais valores não representam convulsão.

In [5]:
# Colocando True onde o valor for igual a 1 e False onde o valor for diferente.
# Esta linha cria uma nova variável no objeto dataset que representa o conjunto de dados.
# Essa variável chama-se 'Diagnostico' e será inserida como uma nova coluna no 
# conjunto de dados. Se a variável da coluna y é igual a 1, então, retorna-se True para
# a nova coluna. Caso não seja igual a 1, o retorno será false.
dataset["Diagnostico"] = dataset.y == 1

In [6]:
# Visualizando alguns registros
dataset.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X171,X172,X173,X174,X175,X176,X177,X178,y,Diagnostico
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-15,-31,-77,-103,-127,-116,-83,-51,4,False
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,150,146,152,157,156,154,143,129,1,True
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,64,48,19,-12,-30,-35,-35,-36,5,False
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-81,-80,-77,-85,-77,-72,-69,-65,5,False
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,2,-12,-32,-41,-65,-83,-89,-73,5,False


In [7]:
# Nesta linha, estamos convertendo o valores booleanos da coluna "Diagnostico" do dataset
# em valores do tipo int. Assim, onde é True, será convertido para 1 e onde é False para 0.
# O método astype(int) faz a conversão automaticamente.
# Essa conversão é obrigatória pois não se trabalha com valores strings ou boleanos, apenas, int.
dataset["Diagnostico"] = dataset["Diagnostico"].astype(int)

In [8]:
# Visualizando os 5 primeiros registros do conjunto de dados dataset.
dataset.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X171,X172,X173,X174,X175,X176,X177,X178,y,Diagnostico
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-15,-31,-77,-103,-127,-116,-83,-51,4,0
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,150,146,152,157,156,154,143,129,1,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,64,48,19,-12,-30,-35,-35,-36,5,0
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-81,-80,-77,-85,-77,-72,-69,-65,5,0
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,2,-12,-32,-41,-65,-83,-89,-73,5,0


In [ ]:
# A coluna original (y) que continha se um paciente está tendo uma convulsão ou não 
# será eliminada, pois era uma variável categórica com 5 status diferentes. 
# Desde então, convertemos isso em uma variável numérica binária chamada Diagnostico. 
# Não precisaremos mais da coluna y. Isso evita multicolinearidade.
dataset.pop("y")

0        4
1        1
2        5
3        5
4        5
        ..
11495    2
11496    1
11497    5
11498    3
11499    4
Name: y, Length: 11500, dtype: int64

In [10]:
# Visualizando alguns registros. Observe a coluna Diagnostico
dataset.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,Diagnostico
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [11]:
# A primeira coluna será descartada devido à sua inutilidade em nosso modelo de aprendizado de máquina. 
# A primeira coluna 'Unnamed' será descartada devido à sua inutilidade em 
# nosso modelo de aprendizado de máquina. 
# A primeira coluna é a de índice zero. O comando axis = 1 significa eixo y
# ou colunas. Se fosse axis = 0 seria linhas.
# o comando inplance = True executa a operação e nada é retornado. Se fosse 
# inplance = False, retornaria uma cópia do objeto.
dataset.drop(dataset.columns[0], axis = 1, inplace = True)

In [12]:
# Visualizando alguns registros
dataset.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,Diagnostico
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [15]:
# Exibindo a quantidade de colunas do dataset.
print("Número de colunas: ", len(dataset.columns))

Número de colunas:  179


In [16]:
# Quantidade de linhas e colunas no dataset
dataset.shape

(11500, 179)

O conjunto de dados tem apenas um recurso exclusivo, e essas são as leituras de EEG. As colunas são divididas para capturar a leitura do EEG em um ponto no tempo e todos os pontos no tempo (todas as 178 colunas) existem no mesmo segundo. 

### Calcular a prevalência da classe positiva

A prevalência é a porcentagem de suas amostras que tem a característica que você está tentando prever. Nesse cenário específico, significa que as pessoas que têm uma convulsão são positivas, enquanto as que não sofrem são negativas. A taxa é calculada por (número de amostras positivas / número de amostras). Portanto, uma taxa de prevalência de 0,2, por exemplo,  significa que 20% de nossa amostra está tendo uma convulsão naquele momento.

In [19]:
# A função a seguir calcula a prevalência da classe positiva (Diagnostico = 1)
# Os valores da coluna "Diagnostico" são passados para função calcula_prevalencia,
# através do parâmetro y_actual. Os valores de y_actual são somados e divididos pelo 
# tamanho da amostra. A função sum() soma apenas os valores 1 porque somar zero
# não altera o resultado.
def calcula_prevalencia(y_actual):
    return sum(y_actual) / len(y_actual)

print("Prevalência da classe Positiva: %.3f" % calcula_prevalencia(dataset["Diagnostico"].values))

Prevalência da classe Positiva: 0.200


O <b>cálculo da prevalência</b> da classe positiva nos informa que 20% dos dados do conjunto de dados foram positivas para pacientes que tiveram convulsão, o que significa que 80% dos dados são de pacientes não tiveram convulsão.

A continuar assim, o modelo de Machine Learning irá aprender muito mais com os 80% dos dados de quem não teve convulsão do que os 20% que tiveram convulsão, ou seja, os dados estão desbalanceados. O ideal seria que estivessem balanceados para que o modelo aprendesse igualmente as características das duas categorias.

## Etapa 3: Preparação dos Dados

Nesta etapa realizamos o processamento de recursos que são categóricos ou ordinais em variáveis numéricas legíveis para o nosso algoritmo de aprendizado de máquina. Por exemplo, variáveis categóricas podem ser processadas com codificação one-hot, ou variáveis ordinais podem ser processadas com codificação de rótulo, por exemplo, a coluna y deletada que variava de 1 a 5,  para que nosso algoritmo de aprendizado de máquina possa entendê-las.

Para o conjunto de dados da epilepsia, existem 178 recursos (colunas), no entanto, uma vez que cada coluna representa um ponto de dados em um ponto específico no tempo e são todas as leituras de EEG, não há necessidade de realizar transformação adicional, porque não há necessidade em alterar ou transformar dados das 178 colunas obtidas por eletroencefalograma.

### Separando as Váriáveis Explicativas (Entrada) da Variável alvo ou tarjet (Saída) para criar um DataFrame

In [ ]:
# Preparando o dataset somente com os dados de interesse para a etapa de modelagem.
# collist é um objeto que recebe do dataset todas as colunas e seus dados

# O objetivo é separar as variáveis de entrada da variável alvo (target).
# Nesta linha estamos selecionando de collist todas as 178 colunas de índice zero a 177, 
# ou seja, 178 colunas e armazenando no objeto cols_input.

# Nesta linha estamos concatenando todas as 178 colunas com a coluna Diagnostico
# e o resultado será armazenado no dataframe df_data.


In [ ]:
# Exibindo as primeiras linhas do DataFrame df_data.


In [ ]:
# Checando se temos colunas duplicadas nos dados de entrada em uma list comprehension.
# Este loop retorna o valor de x para x que percorre a lista cols_input e verifica se
# cada nome de coluna, na lista cols_df_data. Em cada iteração, o nome da coluna atual 
# é atribuído à variável x. Se for, significa que essa coluna aparece mais de uma vez 
# na lista, indicando que é uma coluna duplicada.

# Se a variável dup_cols for igual a zero, significa que não há colunas duplicadas.

# len(dup_cols) == 0: Esta é a condição que o assert está verificando. Ela verifica se 
# o número de colunas duplicadas (ou seja, o comprimento da lista dup_cols) é igual a zero. 
# Em outras palavras, está testando se não há colunas duplicadas.
# Se não tiver, um assertion error será retornado.


In [ ]:
# Checando se temos colunas duplicadas no dataset final. Neste caso, com a
# coluna Diagnostico acrescentada em df_data.



## Etapa 4: Modelagem

### Divisão dos dados em treino, validação e teste

Na análise de dados para Machine Learning, a divisão dos dados em conjuntos de treino, validação e teste é uma etapa crucial. O principal objetivo dessa divisão é garantir que o modelo de Machine Learning seja capaz de generalizar bem para dados que ele nunca viu antes, e não apenas "decorar" os dados que foram usados para treiná-lo.

Vamos entender a função de cada um:

<b>1. Conjunto de Treino</b> (Training Set)
Propósito: Este é o maior conjunto de dados e é usado para treinar o modelo de Machine Learning. O algoritmo aprende padrões e relações nos dados do conjunto de treino, ajustando seus parâmetros internos para minimizar o erro entre as previsões e os valores reais.

<b>Como funciona:</b> É como um aluno estudando para uma prova. Ele usa este conjunto para aprender a matéria, entender os conceitos e praticar resolvendo problemas.

<b>Importância:</b> Um conjunto de treino robusto e representativo permite que o modelo capture as características essenciais dos dados e desenvolva uma compreensão abrangente do problema.

<b>2. Conjunto de Validação/</b> (Validation Set)
Propósito: O conjunto de validação é usado para ajustar os hiperparâmetros do modelo e para avaliar o desempenho do modelo durante o processo de treinamento. Ele ajuda a evitar o overfitting (quando o modelo se adapta demais aos detalhes dos dados de treino, perdendo a capacidade de generalizar) e o underfitting (quando o modelo não consegue capturar os padrões essenciais dos dados).

<b>Como funciona:</b> Continuando a analogia da prova, este conjunto é como fazer exercícios de revisão ou simulados. O aluno verifica se está aprendendo corretamente, ajusta sua estratégia de estudo (hiperparâmetros) com base nos resultados e identifica áreas onde precisa melhorar, sem ver as perguntas da prova final.

<b>Importância:</b> Permite comparar diferentes modelos ou configurações de um mesmo modelo e escolher a melhor versão antes da avaliação final. Se você não usar um conjunto de validação, corre o risco de ajustar seus hiperparâmetros com base no desempenho do conjunto de teste, o que "contaminaria" o conjunto de teste e tornaria sua avaliação final tendenciosa.

<b>3. Conjunto de Teste </b>(Test Set)
Propósito: O conjunto de teste é usado para a avaliação final do desempenho do modelo. Ele consiste em dados completamente novos, que o modelo nunca viu durante o treinamento ou a validação.

<b>Como funciona:</b> Este é o dia da prova final! O modelo é testado com dados totalmente desconhecidos para simular como ele se comportará no "mundo real". A performance neste conjunto é a medida mais honesta e imparcial da capacidade de generalização do modelo.

<b>Importância:</b> Fornece uma estimativa não viesada do desempenho do modelo em dados não vistos. É crucial que o conjunto de teste seja mantido totalmente separado e só seja usado uma única vez, no final do processo, para evitar qualquer otimização acidental do modelo para ele.

<b>Proporções Comuns e Melhores Práticas:</b><br>
As proporções para a divisão dos dados podem variar dependendo do tamanho total do seu conjunto de dados e da complexidade do problema. Algumas proporções comuns são:

Para conjuntos de dados menores: 60% treino, 20% validação, 20% teste, ou, 70% treino, 15% validação, 15% teste.

Para conjuntos de dados maiores: 80% treino, 10% validação, 10% teste (ou até 98% treino, 1% validação, 1% teste para datasets muito grandes).

<b> Melhores práticas:</b><br>

<b>Aleatoriedade:</b> A divisão dos dados deve ser feita de forma aleatória (a menos que haja um componente temporal, onde a ordem é importante, como em séries temporais). Isso garante que cada conjunto seja representativo da distribuição geral dos dados.

<b>Estratificação:</b> Para problemas de classificação com classes desbalanceadas, é importante estratificar a divisão para garantir que cada conjunto tenha uma proporção similar de cada classe.

Ao seguir essas práticas, você aumenta a confiança de que seu modelo de Machine Learning não é apenas bom para os dados que ele viu, mas também para os dados novos e desconhecidos que encontrará no futuro.

### Geração de Amostras Aleatórias

In [ ]:
# Gerando amostras aleatórias dos dados de todo o DataFrame df_data.
# O método sample() embaralha todas as amostras e armazena de volta em df_data.


# Ajustando os índices do dataset ou reindexando os índices em df_data

# df_data.head()

<b>A Importância de Gerar Amostras Aleatórias (Embaralhamento)</b>

A importância de embaralhar as linhas de um conjunto de dados, especialmente antes de dividi-lo em conjuntos de treino, validação e teste, é crucial para garantir que seu modelo de aprendizado de máquina seja treinado e avaliado de forma imparcial e representativa.

<b>Aqui estão os principais motivos:</b>

<b>Evitar Viés Posicional ou Temporal:</b> Muitos datasets são coletados ou organizados de forma sequencial. Por exemplo, dados de um experimento podem estar agrupados por condições. Se você dividisse o dataset sem embaralhar, seu conjunto de treino poderia conter apenas um tipo de dado (por exemplo, as primeiras horas de um dia, ou apenas um grupo experimental), e seu conjunto de teste conteria outro tipo. Isso levaria a um modelo que aprendeu padrões específicos daquele segmento de dados e falharia ao generalizar para outras partes do dataset.
    
<b>Garantir Representatividade:</b> Ao embaralhar, você assegura que cada subconjunto (treino, validação, teste) seja uma amostra aleatória representativa de todo o dataset. Isso significa que a distribuição das classes (em problemas de classificação) ou dos valores (em problemas de regressão) será aproximadamente a mesma em todos os conjuntos, o que é fundamental para um treinamento eficaz e uma avaliação justa.
    
<b>Melhorar a Generalização do Modelo</b> Um modelo treinado em um dataset embaralhado é menos propenso a memorizar a ordem ou a sequência dos dados de treino. Em vez disso, ele é forçado a aprender padrões mais robustos e generalizáveis, pois cada lote de treino que ele vê é composto por exemplos diversos de todo o dataset.


### Separando os conjuntos de treino, validação e teste em 70% 15% 15%

In [ ]:
# Gera um índice para a divisão
# Neste caso estamos coletando aleatoriamente uma fração de 30% do total das 
# amostras e armazenando no DataFrame df_valid_teste
# 70% dos dados, serão destinados ao treinamento do algoritmo.

# Mostra o tamanho da divisão para teste de validação.


In [ ]:
# Fazendo a divisão. Criando um DataFrame para testes, um DataFrame para validação 
# e um DataFrame para treino do algoritmo.
# Divisão em 70/15/15
# Dados de teste. 
# Coletando aleatoriamente 50% dos dados do DataFrame df_valid_Teste (de 30% de amostras). 
# Não sabemos que dados são estes.


# Dados de validação. 
# Para criar o DataFrame df_valid, temos que excluir de df_valid_teste os índices de df_teste 
# destinados a testes. 
# Sobram os registros para df_valid, ou seja, os outros 50% do total de 30% de amostras.


# Dados de treino
# Para criar o DataFrame de treino, retiramos de df_data, os indices de df_valid_teste que são 
# 30% das amostras. Restam 70%.
# O conjunto de dados foi dividido em 70/15/15


<b>Explicação da divisão em dados de treino, validação e teste</b>

Dividindo o df_valid_teste em Teste e Validação

Agora, com o df_valid_teste (que é 30% do total), você precisa dividi-lo igualmente para obter 15% para validação e 15% para teste.

Criando o Conjunto de Teste (15%)
Você faz isso com a primeira linha que você mostrou:

Python

df_teste = df_valid_teste.sample(frac = 0.5)
Explicação:

Você pega o df_valid_teste (que corresponde a 30% do total).

Você usa .sample(frac=0.5) para selecionar aleatoriamente 50% das linhas de df_valid_teste.

Essas 50% das linhas de df_valid_teste se tornam seu df_teste.

Cálculo: Se df_valid_teste é 30% do total, e df_teste é 50% de df_valid_teste, então df_teste é 0.5 * 0.30 = 0.15, ou seja, 15% do df_data original.

Criando o Conjunto de Validação (15%)
E finalmente, você cria o conjunto de validação com a segunda linha que você apresentou:

Python

df_valid = df_valid_teste.drop(df_teste.index)
Explicação:

Você pega o df_valid_teste novamente.

Você usa .drop(df_teste.index) para remover as linhas que já foram selecionadas para o df_teste.

As linhas restantes em df_valid_teste (que são os outros 50% não selecionados para df_teste) formam o seu df_valid.

Cálculo: Da mesma forma, se df_valid_teste é 30% do total, e df_valid é os 50% restantes depois de tirar o df_teste, então df_valid também é 0.5 * 0.30 = 0.15, ou seja, 15% do df_data original.

Resumo da Divisão
Ao final dessas operações, você terá os seguintes conjuntos de dados, com as proporções desejadas:

df_treino: Contém 70% do df_data original. Usado para treinar seu modelo.

df_valid: Contém 15% do df_data original. Usado para ajustar hiperparâmetros e monitorar o desempenho do modelo durante o treinamento (para evitar overfitting).

df_teste: Contém 15% do df_data original. Usado para uma avaliação final e imparcial do desempenho do modelo em dados totalmente não vistos.

### Verificando a Prevalência em cada Subconjunto de Dados

In [ ]:
# Verifique a prevalência de cada subconjunto. 
# A prevalência foi calculada como 0.2 ou 20%, ou seja, 20% das pessoas estão tendo convulsão.
# Checagem de como os dados estão distribuidos:
# Temos que ter uma distribuição similar entre teste, validação e treino.
# Assim, teremos a proporção de 20% para cada subconjunto. O mesmo comportamento dos 
# dados do dataset original deverá ser reproduzido nos seus subconjuntos.
# Serão exibidos com tres casas decimais em cada exibição.
print(
    "Teste(n = %d): %.3f"
    # Será passado para a função calcula_prevalencia os valores da coluna 
    # Diagnostico do DataFrame df_teste.
    
)
print(
    "Validação(n = %d): %.3f"
    # Será passado para a função calcula_prevalencia os valores da coluna 
    # Diagnostico do DataFrame df_valid.
    
)
print(
    "Treino(n = %d): %.3f"
    # Será passado para a função calcula_prevalencia os valores da coluna 
    # Diagnostico do DataFrame df_treino.
   
)

<b>Explicação da Prevalência para Teste, Validação e Treino</b><br>
O resultado para cada subconjunto deverá estar próximo a 0.2, seja, 20%

Teste(n = 1725): 0.204
    
Validação(n = 1725): 0.212

Treino(n = 8050): 0.197

In [ ]:
# Verificando a quantidade de amostras do dataset original depois da divisão em subconjuntos.
# Deverá ter o mesmo número dos dados originais.


# o DataFrame df_data deverá ter a quantidade de dados igual à somatória da quantidade
# de dados de todos os outros DataFrames, ou seja, df_teste, df_valid e df_treino.
# Se não for igual, algo saiu errado.


<b>Balanceamento de Cargas</b>

Queremos equilibrar nosso conjunto de dados para evitar a criação de um modelo em que ele classifique incorretamente as amostras como pertencentes à classe majoritária. Por exemplo, se tivermos um conjunto de dados de detecção de fraude e a maioria dos casos for "Não é fraude", o modelo de classificação binária tenderia a favorecer a classe "Não é fraude", o que leva a resultados enganosos.

Equilibramos nosso conjunto de dados para que as proporções de cada classe sejam as mesmas para evitar a criação de um modelo "burro".

Como temos 2300 amostras positivas, podemos usar o método de balanceamento de subamostras para usar apenas um subconjunto aleatório das amostras negativas.

In [ ]:
# Balanceamento do conjunto de dados

import numpy as np

# Cria um índice através de uma Serie Pandas.
# Serie é um array unidimensional que contém um array de dados e um array de labels chamado índice.
# Automaticamente, o pandas cria o índice para a Serie caso o programador não crie este índice.
# Neste caso, rows_pos receberá as posições ou apenas os índices onde Diagnostico seja igual a 1, 
# ou seja, quem tem convulsão, utilizando o DataFrame df_treino.


# Define valores positivos e negativos do índice
# df_treino.loc[rows_pos] localiza os registros positivos para quem tem convulsão e os armazena 
# em df_treino_pos


# Valor mínimo


# Obtém valores aleatórios para o dataset de treino







<b>Detalhando Cada Linha de Código</b><br>
O balanceamento de classes é uma etapa fundamental quando você tem um conjunto de dados desbalanceado, como neste caso, onde apenas 20% dos registros são positivos (propensos a convulsões). Se não balancearmos, o modelo pode tender a prever a classe majoritária, ignorando a minoritária, pois a "aprendizagem" se inclina para a classe mais frequente.

As linhas apresentadas implementam uma técnica comum de balanceamento chamada <b>undersampling</b> da classe majoritária, garantindo que o número de amostras positivas e negativas seja o mesmo no conjunto de treinamento.
Apenas o dataframe de treino será utilizado.

<b>Vamos analisar cada linha e seu propósito:</b><br>

<b>rows_pos = df_treino.Diagnostico == 1</b><br>

Est<b>a linha cria uma série booleana (True/False) chamada rows_pos. Ela verifica, para cada linha no DataFrame df_treino, se o valor da coluna Diagnostico é igual a 1 (ou seja, se a amostra pertence à classe positiva).

<b>Exemplo:</b> Se a primeira linha tiver Diagnostico como 0, rows_pos terá False para aquela posição. Se a segunda linha tiver Diagnostico como 1, rows_pos terá True para aquela posição.

<b>print(rows_pos)</b>

Esta linha simplesmente imprime a série booleana rows_pos no console. É útil para verificar o resultado da linha anterior e entender como as amostras estão sendo identificadas como positivas ou negativas. Você verá uma longa lista de True e False.

<b>df_train_pos = df_treino.loc[rows_pos]</b>

Aqui, um novo DataFrame chamado <b>df_train_pos</b> é criado. Ele contém apenas as linhas de <b>df_treino</b> onde <b>rows_pos</b> é True. Em outras palavras, <b>df_train_pos</b> armazena todas as amostras do conjunto de treinamento que pertencem à classe positiva (propensos a convulsões).
   
<b>df_train_pos = df_treino.loc[~rows_pos]</b>
    
Similarmente, esta linha cria o DataFrame <b>df_train_neg</b>. O operador ~ (til) inverte a série booleana <b>rows_pos</b>. Assim, <b>~rows_pos</b> será True onde <b>rows_pos</b> era False, e vice-versa. Isso significa que <b>df_train_neg</b> conterá todas as amostras do conjunto de treinamento que pertencem à classe negativa (não propensos a convulsões).

<b>n = np.min([len(df_train_pos), len(df_train_neg)])</b>

Esta é uma linha crucial para o balanceamento. Ela calcula o número de amostras da classe minoritária. No seu caso, a classe minoritária é a positiva (2300 registros, 20%). A classe negativa é a majoritária (11500 - 2300 = 9200 registros, 80%). Portanto, n será igual a len(df_train_pos), que é 2300. Esse n será o tamanho que tanto a classe positiva quanto a negativa terão no conjunto de treino final balanceado.
    
<b>df_treino_final = pd.concat([df_train_pos.sample(n=n, random_state=69),
                             df_train_neg.sample(n=n, random_state=69)],
                            axis=0,
                            ignore_index=True)</b>

<b>Esta linha realiza o balanceamento propriamente dito:</b>

<b>df_train_pos.sample(n=n, random_state=69)</b>: Seleciona n (2300) amostras aleatoriamente do DataFrame de amostras positivas (df_train_pos). O random_state=69 garante que a seleção seja reprodutível; se você rodar o código novamente, as mesmas 2300 amostras serão selecionadas.

<b>df_train_neg.sample(n=n, random_state=69)</b>: Seleciona n (2300) amostras aleatoriamente do DataFrame de amostras negativas (df_train_neg). Aqui é onde o undersampling acontece: de 9200 amostras negativas, apenas 2300 são selecionadas.

<b>pd.concat([...], axis=0, ignore_index=True)</b>: Combina (concatena) as 2300 amostras positivas e as 2300 amostras negativas em um único novo DataFrame chamado df_treino_final.

<b>axis=0</b>: Indica que a concatenação deve ser feita por linhas (uma abaixo da outra).

<b>ignore_index=True</b>: Reinicia os índices do novo DataFrame df_treino_final, evitando que índices duplicados ou sobrepostos das amostras originais causem problemas.

Após esta linha, df_treino_final terá um total de 2 * n (4600) registros, com 2300 amostras positivas e 2300 amostras negativas, ou seja, um <b>balanceamento perfeito</b>.

<b>df_treino_final = df_treino_final.sample(n = len(df_treino_final), random_state=69).reset_index(drop = True)</b>

Neste ponto, <b>df_treino_final</b> tem 2300 amostras positivas seguidas por 2300 amostras negativas (devido ao concat). Isso pode introduzir um viés no treinamento se o modelo aprender que as primeiras 2300 sempre são positivas e as próximas 2300 sempre são negativas.


Esta linha embaralha completamente a ordem das linhas em <b>df_treino_final</b>.

<b>df_treino_final.sample(n = len(df_treino_final), random_state=69)</b>: Seleciona todas as linhas de df_treino_final aleatoriamente. Novamente, <b>random_state=69</b> garante a reprodutibilidade.

<b>.reset_index(drop = True)</b>: Após embaralhar, os índices originais podem estar fora de ordem. Este método redefine o índice do DataFrame para uma sequência numérica padrão (0, 1, 2, ...). drop=True garante que a coluna de índice antiga não seja adicionada como uma nova coluna no DataFrame.

<b>print('Balanceamento em Treino(n = %d): %.3f'%(len(df_treino_final), calcula_prevalencia(df_treino_final.Diagnostico.values)))</b>

<b>Esta linha final imprime uma mensagem para confirmar o resultado do balanceamento:</b>

<b>len(df_treino_final)</b>: Retorna o número total de registros no DataFrame de treino final balanceado (que será 4600).

<b>calcula_prevalencia(df_treino_final.Diagnostico.values)</b>: Assume que calcula_prevalencia é uma função definida em outro lugar do seu código que calcula a proporção da classe positiva (target == 1) em relação ao total de amostras. Para um conjunto perfeitamente balanceado, o resultado deve ser 0.5 (ou 50%). O %.3f formata o número com três casas decimais.

Em resumo, essas linhas de código garantem que seu modelo de Machine Learning seja treinado em um conjunto de dados onde ambas as classes (positiva e negativa) têm o mesmo peso, evitando que o modelo seja viesado em direção à classe majoritária. Isso é crucial para que o modelo aprenda a identificar corretamente ambos os tipos de pacientes (propensos e não propensos a convulsões), especialmente a classe minoritária, que geralmente é a de maior interesse em cenários médicos.

### Salvando DataFrames em Disco

In [ ]:
# Salvamos todos os datasets em disco no formato csv.
# Salva-se todos os datasets em disco no formato csv para não perder 
# o que foi feito até o momento.
# O método to_csv() faz esta operação.




# Salvamos os dados de entrada (colunas preditoras) para facilitar a utilização mais tarde
# A biblioteca pickle converte os dados da lista cols_input em arquivo binario, ou seja, 
# faz a serialização do objeto cols_input.



### Criando Matrizes

In [ ]:
# Cria as matrizes X e Y
# X representam os dados de entrada de dados e Y os dados de saida.
# Isto é feito porque o algoritmo de machine learning trabalha com matrizes.
# Atenção: não podemos utilizar os DataFrames do pandas em machine learning. 
# Temos que convertê-las em matrizes ou arrays do python
# X => Representa os dados de entrada
# Criando em X os dados de entrada para treino e dados de entrada para validação.
# X_treino coleta os valores das colunas de entrada do DataFrame df_treino_final
# cols_input contem as 178 colunas (0 a 177) do dataset.
# X



# Y



# Print



In [ ]:
# Mostra os dados da matriz x_treino apenas para ver como os dados estão dispostos.
# Eles estão no formato de um array numpy porque o modelo de machine learning exige.


### Normalização de Dados

In [ ]:
# Prepara o objeto para normalizar os dados.
# Normalizar dados significa colocar os dados numa mesma escala de valores para ESTE MODELO,
# porque os dados das colunas estão em várias escalas diferentes.
# Apenas os valores de X_treino serão normalizados porque os valores de Y_treino são binários (0 ou 1)
# Para saber se devemos normalizar os dados, devemos consultar a documentação do algoritmo.
# Machine Learning é operação de matrizes. Por isso temos que converter os dados
# dos DataFrames do pandas para matrizes.
# Importando a classe StandardScaler da biblioteca sklearn.
from sklearn.preprocessing import StandardScaler

# Instancinado o objeto na classe StanderScaler()


# Faço o fit
# método fit() basicamente calcula os parâmetros de aprendizagem do modelo


# Salva o objeto em disco e carrega para usamos adiante


import pickle
# A biblioteca pickle faz a serialização do objeto e grava os dados em arquivo
# de disco o objeto scalerfile


# Abre o arquivo para leitura em disco e carrega os dados para o objeto scaler para 
# ser utilizado pelo programa.


# Aplica a normalização em nossas matrizes de dados
# A normalização é a conversão dos dados numa mesma escala exigida pelo algoritmo 
# de machine learning. Este é o trabalho do cientista de dados.



In [ ]:
# Observe a diferença dos dados normalizados exibidos aqui para o dataset original do inicio do projeto.
X_treino_tf

### Construção do Modelo

Funções auxiliares.

In [ ]:
# Importa as funções necessárias
# Serão criados 3 modelos para podermos compará-los se escolher 1, ou seja, 
# o melhor que se adequou aos dados fornecidos.
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

# Função para calcular a especificidade



# Função para gerar relatório de métricas








Como equilibramos nossos dados de treinamento, vamos definir nosso limite em 0,5 para rotular uma amostra prevista como positiva.

In [ ]:
# Limite


<b>O que é Especificidade?</b>

A especificidade é uma métrica crucial em problemas de classificação, especialmente em diagnósticos e previsões onde o objetivo é identificar corretamente a ausência de uma condição. No seu caso, a condição é a disposição para convulsões epiléticas.

Pense assim:

<b>Verdadeiros Negativos (VN)</b>: São os pacientes que não têm disposição para convulsões e o seu modelo previu corretamente que eles não teriam.

<b>Falsos Positivos (FP)</b>: São os pacientes que não têm disposição para convulsões, mas o seu modelo previu, incorretamente, que eles teriam.

A especificidade mede a proporção de pacientes que realmente não têm a condição e foram corretamente identificados como não tendo. Em outras palavras, é a capacidade do seu modelo de evitar falsos alarmes. Uma alta especificidade é importante quando um falso positivo (prever que alguém terá convulsão quando não terá) pode levar a tratamentos desnecessários, ansiedade ou efeitos colaterais.

A fórmula da especificidade é:

especificidade = Verdadeiros Negativos / (Verdadeiros Negativos + Falsos Positivos)

<b>Entendendo a Função calc_specificity</b>:

Agora vamos detalhar cada parte da função:

<b>def calc_specificity(y_actual, y_pred, thresh):
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)</b>

def calc_specificity(y_actual, y_pred, thresh):

Esta linha define a função chamada <b>calc_specificity</b>.

<b>y_actual</b>: Representa os valores reais/verdadeiros do seu dataset. No seu contexto, <b>y_actual</b> contém 0 para pacientes que não têm disposição para convulsões e 1 para pacientes que têm disposição.

<b>y_pred</b>: Representa as previsões feitas pelo seu modelo. Essas previsões são geralmente probabilidades ou pontuações de risco (valores contínuos) que o modelo atribui a cada paciente de ter ou não a condição.

<b>thresh (limite/threshold)</b>: É um valor de corte. Como <b>y_pred</b> são probabilidades ou pontuações, você precisa de um <b>thresh</b> para converter essas previsões contínuas em classificações binárias (0 ou 1). 

Se a previsão <b>y_pred</b> for menor que <b>thresh</b>, o modelo classifica como 0 (não tem a condição). Se for maior ou igual, classifica como 1 (tem a condição).

<b>return sum((y_pred < thresh) & (y_actual == 0)) / sum(y_actual == 0)</b>

<b>Esta linha calcula e retorna o valor da especificidade</b>. Vamos quebrar isso:

<b>y_actual == 0</b>: Esta parte seleciona todos os pacientes que realmente não têm a disposição para convulsões. No seu dataset, onde 20% são positivos e o restante negativos, y_actual == 0 identificará as 9200 linhas (80% de 11.500) que são negativas.

<b>sum(y_actual == 0)</b>: O sum() aqui conta quantos pacientes realmente não têm a disposição para convulsões. Este é o <b>denominador</b> da sua fórmula de especificidade, representando <b>Verdadeiros Negativos + Falsos Positivos</b>.

<b>(y_pred < thresh)</b>: Esta parte identifica as previsões do seu modelo onde a pontuação/probabilidade foi abaixo do limiar. Isso significa que o modelo previu que esses pacientes não têm a condição.

<b>(y_pred < thresh) & (y_actual == 0)</b>: Esta é a parte mais importante. O operador <b>&</b> (AND lógico) combina as duas condições. Ele seleciona apenas os casos onde <b>ambas as condições são verdadeiras</b>:

O modelo previu que o paciente não tem a condição <b>(y_pred < thresh)</b>.
O paciente realmente não tem a condição <b>(y_actual == 0)</b>.
Esta combinação identifica os <b>Verdadeiros Negativos (VN)</b>.

<b>sum((y_pred < thresh) & (y_actual == 0))</b>: O sum() aqui conta quantos Verdadeiros Negativos foram identificados. Este é o numerador da sua fórmula de especificidade.

Em resumo, a função está calculando:
    
<b>Especificidade = Número de Verdadeiros Negativos / Número Total de Pacientes que Não têm a condição</b>

Isso é exatamente o que a especificidade representa!

<b>Função print_report - Métricas Calculadas e seus Significados</b>

Vamos entender o que cada uma dessas métricas representa no contexto do seu projeto de previsão de convulsões epiléticas:

<b>AUC (Area Under the Receiver Operating Characteristic Curve)</b>

<b>Linha de Código: auc = roc_auc_score(y_actual, y_pred)</b>

<b>Significado</b>: O AUC é uma das métricas mais robustas para avaliar a performance de um modelo de classificação, especialmente quando ele produz probabilidades ou pontuações de risco (como o seu y_pred). Ele mede a capacidade do modelo de distinguir entre as classes positiva e negativa em todos os possíveis limiares de classificação.

Um AUC de 1.0 significa que o modelo é perfeito, classificando corretamente todos os pacientes.

Um AUC de 0.5 sugere que o modelo não é melhor do que um palpite aleatório.

No seu caso, um AUC alto indica que o modelo é bom em diferenciar pacientes com e sem disposição para convulsões, independentemente do limiar escolhido para fazer a classificação final. É particularmente útil em datasets desbalanceados, como o seu (20% positivos vs. 80% negativos), pois não é sensível à distribuição das classes.

<b>Acurácia (Accuracy)</b>

<b>Linha de Código:</b> accuracy = accuracy_score(y_actual, (y_pred > thresh))

<b>Significado</b>: A acurácia mede a proporção de previsões corretas (tanto positivos quanto negativos) em relação ao total de previsões.

<b>y_pred > thresh</b>: Aqui, suas previsões de probabilidade (y_pred) são convertidas em classificações binárias (0 ou 1) usando o thresh. Se a probabilidade for maior que thresh, o modelo prevê 1 (disposição para convulsões); caso contrário, prevê 0 (sem disposição).

A acurácia é intuitiva e fácil de entender: ela diz quantos pacientes o modelo classificou corretamente no geral. No entanto, em datasets desbalanceados (como o seu, onde 80% são negativos), uma alta acurácia pode ser enganosa. Um modelo que sempre prevê "não tem convulsão" já acertaria 80% dos casos, mesmo sem aprender nada de útil. Por isso, é crucial olhar outras métricas junto com a acurácia.

<b>Recall (Sensibilidade ou True Positive Rate)</b>

<b>Linha de Código:</b> recall = recall_score(y_actual, (y_pred > thresh))

<b>Significado</b>: O recall (também conhecido como sensibilidade) mede a proporção de casos positivos reais que foram corretamente identificados pelo modelo.

No seu contexto, o recall diz quantos dos pacientes que realmente têm disposição para convulsões foram identificados como tendo essa disposição pelo seu modelo.

Um recall alto é fundamental quando o custo de um Falso Negativo é alto (ou seja, não identificar um paciente que tem disposição para convulsões). Não detectar um paciente com risco pode ter consequências sérias. Um bom recall minimiza esses "desvios" ou "perdas" de casos positivos.

<b>Precisão (Precision)</b>

<b>Linha de Código:</b> precision = precision_score(y_actual, (y_pred > thresh))

<b>Significado</b>: A precisão mede a proporção de previsões positivas que foram realmente corretas.

No seu projeto, a precisão responde: "Dos pacientes que o meu modelo previu que teriam convulsões, quantos realmente tiveram?"

Uma alta precisão é importante quando o custo de um Falso Positivo é alto (ou seja, classificar um paciente como tendo disposição para convulsões quando ele não tem). Isso poderia levar a tratamentos desnecessários, ansiedade ou exames adicionais caros.

<b>Especificidade (Specificity)</b>

<b>Linha de Código:</b> specificity = calc_specificity(y_actual, y_pred, thresh)

<b>Significado</b>: A especificidade mede a proporção de casos negativos reais que foram corretamente identificados pelo modelo.

Ela responde: "Dos pacientes que realmente não têm disposição para convulsões, quantos o meu modelo corretamente identificou como não tendo?"

É o oposto do recall, focando nos negativos. Uma alta especificidade minimiza os Falsos Positivos, o que é crucial se você quer evitar intervenções desnecessárias.

<b>Por que todas essas métricas?</b>

Nenhuma métrica isolada conta toda a história. Em um projeto de saúde, especialmente com dados desbalanceados, você precisa de um equilíbrio entre elas, dependendo do que é mais crítico:

Se é mais importante não perder nenhum paciente com risco (evitar Falsos Negativos), você focaria mais no Recall.

Se é mais importante evitar alarmes falsos e não submeter pacientes saudáveis a procedimentos desnecessários (evitar Falsos Positivos), você olharia mais para a Precisão e Especificidade.

O AUC dá uma visão geral da capacidade discriminatória do modelo, independentemente do thresh.

A Acurácia é um bom ponto de partida, mas precisa ser interpretada com cautela em datasets desbalanceados.

Ao usar essa função print_report, você tem um panorama completo do desempenho do seu modelo, permitindo tomar decisões informadas sobre o melhor thresh e as próximas etapas no desenvolvimento do modelo.

### Criação do Modelo

Serão criados três modelos para este estudo. São eles:

1 - Regressão Logística

2 - Naive Bayes

3 - XGBoost

Ao final devemos escolher qual deles será o mais eficiente!

### Modelo 1 - Regressão Logística

A Regressão Logística é um algoritmo de aprendizado de máquina usado para problemas de classificação, e não de regressão, apesar do nome. Ela é amplamente utilizada para prever a probabilidade de um evento acontecer, ou seja, para classificar observações em uma de duas categorias (classificação binária), ou em múltiplas categorias (classificação multinomial).

Como a Regressão Logística Funciona?
Ao contrário da regressão linear, que prevê um valor contínuo, a regressão logística utiliza uma função sigmoide (ou logística) para transformar a saída linear em uma probabilidade. Essa probabilidade é então mapeada para uma classe.

Vamos detalhar os componentes principais:

Combinação Linear das Entradas:

Assim como na regressão linear, a regressão logística começa calculando uma combinação linear das variáveis de entrada (características) multiplicadas por seus respectivos pesos (coeficientes), somando um termo de viés (intercepto). Seus dados de entrada, como as 178 colunas do seu dataset, seriam as variáveis que alimentam essa combinação.

A fórmula é:

z = β0 + β1x1 + β2x2 + ... + βnxn

<b> Função Sigmoide (ou Logística):</b>

O resultado z da combinação linear não é uma probabilidade e pode variar de −∞ a +∞. Para transformar z em uma probabilidade que esteja entre 0 e 1, a regressão logística aplica a função sigmoide:

P(Y=1∣X) = 1 / (1 + e^-z)

Onde:

P(Y=1∣X) é a probabilidade de a variável dependente (Y) ser 1 (a classe positiva) dadas as variáveis de entrada (X).

e é o número de Euler (base do logaritmo natural).

O gráfico da função sigmoide tem um formato de "S", comprimindo qualquer valor de z para um valor entre 0 e 1.

<b>Classificação:</b>

Após obter a probabilidade, um limiar (threshold) é aplicado para decidir a qual classe a observação pertence. Por exemplo, se a probabilidade calculada for maior que 0.5 (o limiar padrão), o modelo classifica a observação como pertencente à classe 1 (positiva); caso contrário, como classe 0 (negativa).

<b>Por que o Nome "Regressão Logística"?</b>

O nome "regressão" deriva do fato de que, fundamentalmente, ela ainda está modelando a relação entre as variáveis de entrada e a saída. No entanto, o "logística" refere-se à utilização da função logística (sigmoide) e ao fato de que ela modela a probabilidade logarítmica (log-odds) de um evento, que é uma função linear das variáveis de entrada.

<b>Aplicações ao Projeto</b>

Neste projeto sobre previsão de convulsões epiléticas, a Regressão Logística seria uma ótima escolha para:

Prever a probabilidade de um paciente ter disposição para convulsões (classe 1) ou não (classe 0).

Identificar quais fatores (colunas do seu dataset) são mais influentes nessa previsão, analisando os pesos (β) atribuídos a cada variável.

É um modelo simples, mas muito eficaz e interpretável, o que o torna popular em diversas áreas, incluindo a saúde.

In [ ]:
# Construção do modelo

# Importa a bilioteca LogisticRegression que contém o algoritmo de Regressão Logística
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Cria o classificador (objeto)
# O parâmetro random_state = 142 permite que os resultados sejam os mesmos em caso de 
# executar o a célula novamente.


# Treina e cria o modelo baseado no classificador lr. O método fit faz o treinamento do 
# modelo X_treino_tf e y_treino que é o array normalizado dos dados de entrada e saida.
# Atenção: Apenas os valores de X foram normalizados porque são vários valores diferentes.
# Os valores da coluna Y não são normalizados porque são apenas dois valores, ou seja, 0 ou 1.


# Previsões 



print('Regressão Logística')


print('Treinamento:')


print('Validação:')



<b>Conclusão:</b> 

Se o objetivo é obter o mais próximo de 99%, significa que o modelo de Regressão Logística não foi uma boa escolha. Embora a acurácia tenha estado próximo a 70%, pode-se observar que outros índices ficarão aquém da espectativa. A curva AUC, por exemplo, obteve 0.627 para treinamento e uma taxa menor de 0.533 para validação.

A discrepância entre a Precisão de treinamento de 0.711 para validação 0.349 é ainda maior.

### Modelo 2 - Naive Bayes

O Naive Bayes é um algoritmo de aprendizado de máquina classificador probabilístico supervisionado. Ele se baseia no famoso Teorema de Bayes e faz uma suposição "ingênua" (daí o "Naive") de independência entre as características (atributos) do seu conjunto de dados. Apesar dessa suposição simplista, que raramente é verdadeira no mundo real, o Naive Bayes é surpreendentemente eficaz e amplamente utilizado em diversas aplicações, como filtragem de spam, classificação de documentos e análise de sentimento.

<b>Como o Naive Bayes Funciona</b>

A essência do Naive Bayes reside em calcular a probabilidade de um determinado item pertencer a uma classe, dadas suas características. Ele faz isso em duas fases principais:

<b>Fase de Treinamento (Aprendizado):</b>

Calcula as Probabilidades a Priori das Classes: Para cada classe (categoria) em seu conjunto de dados de treinamento, o algoritmo determina a probabilidade de um item qualquer pertencer a essa classe. Por exemplo, se você está classificando e-mails como "Spam" ou "Não Spam", ele calcularia qual porcentagem dos e-mails de treinamento são "Spam" e qual porcentagem são "Não Spam".

Calcula as Verossimilhanças (Probabilidades Condicionais): Para cada característica e para cada valor que essa característica pode assumir, o Naive Bayes calcula a probabilidade de essa característica ter aquele valor, dado que o item pertence a uma classe específica. Por exemplo, qual é a probabilidade de um e-mail conter a palavra "promoção" se ele for um "Spam"? E se ele for um "Não Spam"? Ele faz isso para todas as palavras (características) e todas as classes.

<b>Fase de Classificação (Predição):</b>

Quando um novo item com características desconhecidas precisa ser classificado, o Naive Bayes usa o Teorema de Bayes para calcular a probabilidade a posteriori de o item pertencer a cada uma das classes disponíveis. A fórmula fundamental é:

P(Classe∣Caracteristicas) ∝ P(Caracteristicas∣Classe)⋅P(Classe)

Graças à sua "suposição ingênua", a probabilidade das características dada a classe P(Caracteristicas∣Classe) 
é simplificada para o produto das probabilidades de cada característica individualmente:

P(Caracteristicas∣Classe) = P(Caracteristica1∣Classe)

P(Caracteristica2∣Classe)⋅⋯⋅P(Caracteristican|Classe) 

O algoritmo então multiplica a probabilidade a priori da classe pelas probabilidades condicionais de cada característica (calculadas na fase de treinamento).

Finalmente, o item é atribuído à classe para a qual o cálculo resultou na maior probabilidade.

<b>A Suposição "Ingênua"</b>
A parte "ingênua" é que o Naive Bayes assume que todas as características são independentes umas das outras, dada a classe. Isso significa que, por exemplo, a presença da palavra "promoção" em um e-mail não afeta a probabilidade de a palavra "grátis" também estar presente, se você já sabe se o e-mail é spam ou não. Na realidade, palavras geralmente têm alguma dependência, mas o algoritmo ainda consegue ser eficaz porque o que realmente importa para a classificação é a ordem relativa das probabilidades, e não suas magnitudes absolutas perfeitas.

<b>Vantagens e Desvantagens</b>

<b>Vantagens:</b>

Simples e Rápido: É fácil de implementar e muito eficiente computacionalmente, o que o torna rápido para treinar e fazer previsões, mesmo com grandes volumes de dados.

Bom Desempenho: Surpreendentemente eficaz em muitos cenários do mundo real, especialmente em problemas de classificação de texto e filtragem de spam.

Lida Bem com Alta Dimensionalidade: Pode lidar eficientemente com um grande número de características.

Requer Poucos Dados de Treinamento: Pode apresentar bons resultados mesmo com um volume moderado de dados, embora se beneficie de mais dados para estimar probabilidades com maior precisão.

<b>Desvantagens:/<b>

<b>A "Suposição Ingênua":</b> A independência das características raramente é verdadeira, o que pode limitar seu desempenho em alguns casos onde as inter-relações entre as características são cruciais.

Problema de Probabilidade Zero: Se uma característica não aparece nos dados de treinamento para uma determinada classe, a probabilidade condicional dessa característica se torna zero, o que pode anular todo o cálculo da probabilidade para aquela classe. Isso é geralmente resolvido com técnicas de suavização, como a Suavização de Laplace (Add-1 Smoothing).

Não é um Bom Estimador de Probabilidade: Embora seja um bom classificador (capaz de identificar a classe correta), as probabilidades que ele gera podem não ser as mais precisas.

O Naive Bayes é uma ferramenta fundamental no kit de ferramentas de qualquer cientista de dados, oferecendo um excelente equilíbrio entre simplicidade, velocidade e desempenho para uma ampla gama de tarefas de classificação.

In [ ]:
# Construção do modelo

# Imports
from sklearn.naive_bayes import GaussianNB

# Cria a instância ao objeto utilizando o construtor da classe GaussianNB


# Treina e cria o modelo


# Previsões



print('Naive Bayes')

# Cálculo das Métricas
print('Treinamento:')


print('Validação:')



<b>Conclusão para Naive Bayes</b>

Apenas a mudança de um algoritmo por outro, resultou numa melhora significativa na performance do nosso estudo. Isso significa que este algoritmo se ajustou melhor para este conjunto de dados do que para o de Regressão Logistica. Isto sem modificar um único parâmetro de configuração. 

<b> Interpretando as duas linhas de Código sobre Previssões: </b>

y_train_preds = nb.predict_proba(X_treino_tf)[:,1]

y_valid_preds = nb.predict_proba(X_valid_tf)[:,1]

Estas linhas de código estão relacionadas ao processo de predição de probabilidades em um modelo Naive Bayes (representado por nb), especificamente obtendo as probabilidades para a segunda classe (índice 1).

Vamos analisar o fatiamento [:,1] em detalhes:

<b>Entendendo nb.predict_proba()</b>

Primeiramente, é importante saber que o método predict_proba() em bibliotecas de aprendizado de máquina (como o scikit-learn, que é o mais comum para Naive Bayes em Python) retorna as probabilidades de um exemplo pertencer a cada uma das classes que o modelo foi treinado para reconhecer.

Se você tem um problema de classificação binária (duas classes, por exemplo, 0 e 1, ou "Não Spam" e "Spam"), predict_proba() retornará uma matriz com duas colunas para cada exemplo:

A primeira coluna (índice 0): representa a probabilidade do exemplo pertencer à primeira classe (por exemplo, a classe negativa, ou classe 0).

A segunda coluna (índice 1): representa a probabilidade do exemplo pertencer à segunda classe (por exemplo, a classe positiva, ou classe 1).

<b>Exemplo de saída de nb.predict_proba():</b>

[[0.9, 0.1],  # Exemplo 1: 90% chance de ser classe 0, 10% chance de ser classe 1
 [0.2, 0.8],  # Exemplo 2: 20% chance de ser classe 0, 80% chance de ser classe 1
 [0.6, 0.4]]  # Exemplo 3: 60% chance de ser classe 0, 40% chance de ser classe 1
 
<b>O que significa o fatiamento [:,1]?/<b>

O fatiamento [:,1] é uma operação comum em arrays NumPy (que é o formato de dados que predict_proba() geralmente retorna) e significa o seguinte:

<b>: (dois pontos)</b>: Indica que queremos selecionar todas as linhas do array. Ou seja, estamos interessados nas probabilidades de todos os exemplos (sejam eles do conjunto de treino ou de validação).

<b>, (vírgula)</b>: Separa a seleção de linhas da seleção de colunas.

<b>1:</b> Indica que queremos selecionar apenas a coluna com o índice 1.

<b>Juntos, [:,1]</b> significa "para todas as linhas, pegue a coluna de índice 1".

<b>Finalidade das linhas de código</b>
    
<b>y_train_preds = nb.predict_proba(X_treino_tf)[:,1]
    
y_valid_preds = nb.predict_proba(X_valid_tf)[:,1]</b>

<b>y_train_preds:</b> Armazenará as probabilidades de cada exemplo no conjunto de treinamento pertencer à segunda classe (a classe "positiva" ou de interesse, que geralmente é a classe 1).

<b>y_valid_preds:</b> Armazenará as probabilidades de cada exemplo no conjunto de validação pertencer à segunda classe (a classe "positiva" ou de interesse).

Essa abordagem é muito comum quando você está interessado na probabilidade de pertencer a uma classe específica (geralmente a classe positiva em problemas binários), especialmente se você for usar essas probabilidades para calcular métricas como a Curva ROC, AUC, ou para definir um limiar de corte personalizado.

<b> Conclusão do Modelo Naive Bayes </b>

Podemos observar que tivemos um ganho significativo apenas substituindo um algoritmo por outro. 

Os valores das métricas de treinamento e validação são praticamente os mesmos.

Isto significa que o algoritmo Naive Bayes é bem melhor que o de regressão logística para este conjunto de dados.

### Modelo 3 - XGBoost (Xtreme Gradient Boosting Classifier)

O Modelo XGBoost (Extreme Gradient Boosting Classifier)

O XGBoost (Extreme Gradient Boosting) é um algoritmo de aprendizado de máquina extremamente popular e poderoso, conhecido por sua alta performance e eficiência em competições de ciência de dados (como o Kaggle) e em aplicações práticas de grande escala. Ele é uma implementação otimizada e escalável da técnica de Gradient Boosting (Aumento de Gradiente).

Para entender o XGBoost, é fundamental compreender alguns conceitos subjacentes:

<b>1. Árvores de Decisão (Decision Trees)</b>

A base do XGBoost são as árvores de decisão. Uma árvore de decisão é um modelo que se assemelha a um fluxograma, onde cada nó interno representa um "teste" em um atributo, cada ramificação representa o resultado desse teste e cada nó folha representa uma decisão final ou um valor numérico. Elas são simples de entender e interpretar, mas modelos baseados em uma única árvore podem ser instáveis e ter baixo poder preditivo.

<b>2. Ensemble Learning (Aprendizado em Conjunto)</b>

O XGBoost é um método de ensemble learning, o que significa que ele combina múltiplas árvores de decisão (modelos "fracos") para criar um modelo "forte" e mais robusto. As duas principais abordagens de ensemble são:

<b>Bagging (Bootstrap Aggregating):</b> Cria várias árvores de forma independente em subconjuntos bootstrap dos dados (ex: Random Forest).

<b>Boosting:</b> Constrói árvores sequencialmente, onde cada nova árvore tenta corrigir os erros das árvores anteriores. O XGBoost se enquadra nesta categoria.

<b>3. Gradient Boosting (Aumento de Gradiente)</b>

O XGBoost é uma evolução do algoritmo Gradient Boosting Machine (GBM). A ideia central do Gradient Boosting é construir o modelo de forma aditiva:

Ele começa com um modelo inicial (muitas vezes, uma predição média ou constante).

Em cada etapa, ele treina uma nova árvore de decisão para prever o resíduo (erro) do modelo combinado até então. Em outras palavras, a nova árvore tenta aprender o que o modelo atual "errou".

Os resíduos são calculados com base no gradiente de uma função de perda. Isso significa que ele usa uma técnica de otimização de gradiente (como o gradiente descendente) para minimizar a função de perda (ex: erro quadrático médio para regressão, log loss para classificação).

As previsões das novas árvores são adicionadas ao modelo existente, geralmente com um pequeno "fator de encolhimento" (learning rate) para evitar o overfitting e garantir que o modelo aprenda gradualmente.

<b>O "Xtreme" no XGBoost:</b> Otimizações e Recursos

O "Extreme" no XGBoost vem das diversas otimizações e recursos que o tornam superior a implementações tradicionais de Gradient Boosting:

Paralelização: O XGBoost é otimizado para paralelização. Embora as árvores sejam construídas sequencialmente, o processo de encontrar os melhores splits (divisões) dentro de cada árvore pode ser paralelizado.

Manuseio de Valores Ausentes: Ele tem uma estratégia embutida para lidar com valores ausentes, o que o torna mais robusto e elimina a necessidade de pré-processamento manual para imputação.

<b>Técnicas de Regularização:</b>

L1 (Lasso) e L2 (Ridge) Regularização: Ajuda a evitar o overfitting penalizando a complexidade do modelo, suavizando as previsões.

Subamostragem de Linhas e Colunas (Colsample_bytree, Subsample): Semelhante ao Random Forest, ele pode usar apenas uma fração aleatória dos dados e/ou das características para construir cada árvore, o que reduz o overfitting e acelera o treinamento.

<b>Otimização de Hardware:</b>

Algoritmo para splits esparsos: Eficiente para dados esparsos.

Alocação de memória: Utiliza blocos de memória para acesso eficiente aos dados, reduzindo o custo computacional.

Cache-aware access: Otimiza o uso do cache da CPU.

Poda (Pruning) da Árvore: Após a construção inicial, o XGBoost poda as árvores para remover ramos que não contribuem significativamente para a redução da função de perda. Isso também ajuda a evitar o overfitting.

Validação Cruzada Integrada: Permite a validação cruzada em cada iteração de boosting, o que facilita o monitoramento do desempenho do modelo e a seleção do número ideal de iterações para evitar o overfitting.

Flexibilidade da Função de Perda: Permite que o usuário defina funções de perda personalizadas, o que o torna aplicável a uma vasta gama de problemas.

Como o XGBoost Funciona para Classificação (XGBClassifier)
Para problemas de classificação, o XGBoost, assim como outros algoritmos de Gradient Boosting, constrói uma série de árvores de decisão que tentam corrigir os erros das árvores anteriores.

<b>Inicialização:</b> Começa com uma predição inicial (geralmente a log-odds da probabilidade da classe positiva para classificação binária).

Cálculo dos Resíduos (Pseudo-Resíduos): Em cada iteração, ele calcula os "pseudo-resíduos", que são o gradiente negativo da função de perda em relação à predição atual do modelo. Essencialmente, ele calcula o quão "errada" a previsão atual é e em que direção ela precisa ser ajustada.

Treinamento de uma Nova Árvore: Uma nova árvore de decisão é treinada para prever esses pseudo-resíduos. A árvore aprende as relações entre as características e os erros do modelo atual.

<b>Adição ao Modelo:</b> A previsão da nova árvore é multiplicada por um "learning rate" (taxa de aprendizado, um hiperparâmetro pequeno como 0.01 a 0.3) e adicionada à predição do modelo combinado. Essa taxa de aprendizado garante que cada árvore faça uma pequena contribuição, permitindo que o modelo aprenda de forma gradual e robusta.

Regularização e Otimização: Durante a construção da árvore, o XGBoost aplica suas técnicas de regularização (L1, L2, subamostragem) e otimizações para evitar overfitting e aumentar a eficiência.

<b>Iteração:</b> O processo se repete por um número predefinido de iterações (número de árvores), ou até que o desempenho em um conjunto de validação não melhore.

Para classificação binária, o resultado final do XGBoost é uma pontuação contínua (logit), que é então passada por uma função sigmoide para obter uma probabilidade entre 0 e 1. Para classificação multiclasse, ele usa uma função softmax para obter probabilidades para cada classe.

<b>Quando Usar XGBoost?</b>

Problemas com dados tabulares: Onde a maioria dos dados são numéricos ou categóricos que podem ser codificados.

Conjuntos de dados de tamanho médio a grande: Brilha em datasets com centenas de milhares ou milhões de exemplos.

Quando a performance é crítica: Em cenários onde alta acurácia é essencial.

Competições de Machine Learning: É um algoritmo de escolha por sua robustez e resultados.

O XGBoost não é apenas um algoritmo; é um framework de software otimizado para implementar o Gradient Boosting de forma eficiente. Sua combinação de técnicas de regularização, otimizações de hardware e flexibilidade o torna uma ferramenta incrivelmente poderosa e versátil para uma ampla gama de problemas de classificação e regressão, elevando o Gradient Boosting a um nível de "extreme performance".

In [ ]:
# Instalando a biblioteca xgboost
# Se não estiver instalada, basta remover o símbolo de comentário # e instalar
# !pip install xgboost

In [ ]:
# Construção do modelo

# Imports
from xgboost import XGBClassifier
import xgboost as xgb

# Cria o classificador


# Treina e cria o modelo


# Previsões



print('Xtreme Gradient Boosting Classifier')


print('Treinamento:')


print('Validação:')



<b>Conclusão para o Modelo XGBoost</b>

Observe que as métricas de AUC e Acurácia para treino e validação neste modelo são superiores ao algoritmo de Naive Bayes. Ou seja, o conjunto de dados em estudo se ajustou ainda melhor a este algoritmo.

## Etapa 5: Avaliação e Interpretação dos Resultados

Vamos criar um dataframe com esses resultados e plotar os resultados usando o seaborn.

In [ ]:
df_results = pd.DataFrame({'classificador':['RL','RL','NB','NB','XGB','XGB'],
                           'data_set':['treino','validação']*3,
                          'auc':[lr_train_auc,lr_valid_auc,nb_train_auc,nb_valid_auc,xgbc_train_auc,xgbc_valid_auc],
                          'accuracy':[lr_train_accuracy,lr_valid_accuracy,nb_train_accuracy,nb_valid_accuracy,xgbc_train_accuracy,xgbc_valid_accuracy],
                          'recall':[lr_train_recall,lr_valid_recall,nb_train_recall,nb_valid_recall,xgbc_train_recall,xgbc_valid_recall],
                          'precision':[lr_train_precision,lr_valid_precision,nb_train_precision,nb_valid_precision,xgbc_train_precision,xgbc_valid_precision],
                          'specificity':[lr_train_specificity,lr_valid_specificity,nb_train_specificity,nb_valid_specificity,xgbc_train_specificity,xgbc_valid_specificity]})

In [ ]:
# Imports e definição do estilo do gráfico no seaborn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style = "whitegrid")

A métrica de desempenho escolhida será a pontuação AUC (AUC Score) do conjunto de validação. É a pontuação mais comum usada para comparar qual modelo é melhor na classificação de amostras.

In [ ]:
# Construção do Plot


# Gráfico de barras



# Legenda


Nosso melhor modelo é o Classificador XGBoost, com uma AUC de validação de 99,1%.

### Gravando o melhor modelo

Escolhemos o modelo_v3 simplesmente porque ele tem a AUC de validação mais alta, pois essa é a métrica que escolhemos para avaliar os modelos.

In [ ]:
# Grava o modelo em disco


### Avaliando o Modelo

In [ ]:
# Carrega o modelo, as colunas e o scaler



# Carrega os dados de treino, validação e teste



# Cria matrizes x e y

# X



# Y



# Aplica a transformação nos dados




Calcular probabilidades de previsão.

Avaliação de desempenho.

Curva ROC.

In [ ]:
# Imports
from sklearn.metrics import roc_curve 

# Calcula a curva ROC nos dados de treino
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_preds)
auc_train = roc_auc_score(y_train, y_train_preds)

# Calcula a curva ROC nos dados de validação
fpr_valid, tpr_valid, thresholds_valid = roc_curve(y_valid, y_valid_preds)
auc_valid = roc_auc_score(y_valid, y_valid_preds)

# Calcula a curva ROC nos dados de teste
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_preds)
auc_test = roc_auc_score(y_test, y_test_preds)

# Plot
plt.figure(figsize=(16,10))
plt.plot(fpr_train, tpr_train, 'r-',label ='AUC em Treino: %.3f'%auc_train)
plt.plot(fpr_valid, tpr_valid, 'b-',label ='AUC em Validação: %.3f'%auc_valid)
plt.plot(fpr_test, tpr_test, 'g-',label ='AUC em Teste: %.3f'%auc_test)
plt.plot([0,1],[0,1],'k--')
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.legend()
plt.show()

## Etapa 6: Deploy do Modelo

In [ ]:
# Carregando dados de um novo paciente


In [ ]:
# Exibindo estes dados do novo paciente


In [ ]:
# Transformando os dados do novo paciente para a mesma escala de treino


In [ ]:
# Exibindo os 178 dados do novo paciente transformados para nova escala


In [ ]:
# Calculando a Predição para os dados do novo paciente utilizando o melhor modelo


Primeiro valor (9.9995697e-01): Este é o valor 0.99995697. 

Ele representa a probabilidade de o paciente pertencer à primeira classe (índice 0). Em um problema de classificação binária, essa geralmente é a classe "negativa" ou a classe que representa a ausência da condição que você está tentando prever (por exemplo, "não doente").

Segundo valor (4.3044005e-05): Este é o valor 0.000043044005. 

Ele representa a probabilidade de o paciente pertencer à segunda classe (índice 1). Esta é geralmente a classe "positiva" ou a presença da condição (por exemplo, "doente").

In [ ]:
# Estabelece a classe o novo paciente se enquadra


<b>array([0]) significa que o novo paciente se enquadra na classe da posição zero do array, ou seja, 99% de chances de não ter convulsão.</b>

De acordo com os dados do exame, esse paciente não terá uma crise epilética.

# Conclusão

Se esse modelo for colocado em produção para prever se um paciente está tendo uma crise epilética, você pode esperar que ele tenha um bom desempenho.

# Fim